# Custom Header Routing with Seldon and Ambassador

This notebook shows how you can deploy Seldon Deployments which can have custom routing via Ambassador's custom header routing.


## Prerequistes
You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core) running this notebook
 - A running Kubernetes cluster with kubectl authenticated
 - [Helm client](https://helm.sh/)

### Creating a Kubernetes Cluster

Follow the [Kubernetes documentation to create a cluster](https://kubernetes.io/docs/setup/).

Once created ensure ```kubectl``` is authenticated against the running cluster.

# Setup

In [ ]:
!kubectl create namespace seldon

In [ ]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

In [ ]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

# Install Helm

In [ ]:
!kubectl -n kube-system create sa tiller
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
!helm init --service-account tiller

In [ ]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

## Start seldon-core

In [ ]:
!helm install ../../../helm-charts/seldon-core-crd --name seldon-core-crd --set usage_metrics.enabled=true

In [ ]:
!helm install ../../../helm-charts/seldon-core --name seldon-core --namespace seldon  --set ambassador.enabled=true

In [ ]:
!kubectl rollout status deploy/seldon-core-seldon-cluster-manager
!kubectl rollout status deploy/seldon-core-seldon-apiserver
!kubectl rollout status deploy/seldon-core-ambassador

## Set up REST and gRPC methods

**Ensure you port forward ambassador**:

```
kubectl port-forward $(kubectl get pods -n seldon -l service=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080
```

## Launch main model

We will create a very simple Seldon Deployment with a dummy model image `seldonio/mock_classifier:1.0`. This deployment is named `example`.

In [ ]:
!pygmentize model.json

In [ ]:
!kubectl create -f model.json

In [ ]:
!kubectl rollout status deploy/production-model-single-7cd068f

### Get predictions

In [2]:
import sys
sys.path.append("../../../notebooks")
from seldon_utils import *
API_AMBASSADOR="localhost:8003"

#### REST Request

In [ ]:
r = rest_request_ambassador("example","seldon",API_AMBASSADOR)
print(r.text)

#### gRPC Request

In [ ]:
grpc_request_ambassador("example","seldon",API_AMBASSADOR)

## Launch Model with Custom Routing

We will now create a new graph for our Canary with a new model `seldonio/mock_classifier_rest:1.1`. To make it a canary of the original `example` deployment we add two annotations

```
"annotations": {
	    "seldon.io/ambassador-header":"location:london"
	    "seldon.io/ambassador-service-name":"example"	    
	},	
```

The first annotation says we want to route traffic that has the header `location:london`. The second says we want to use `example` as our service endpoint rather than the default which would be our deployment name - in this case `example-canary`. This will ensure that this Ambassador setting will apply to the same prefix as the previous one.

In [ ]:
!pygmentize model_with_header.json

In [ ]:
!kubectl create -f model_with_header.json

In [ ]:
!kubectl rollout status deploy/header-model-single-4c8805f

Check a request without a header goes to the existing model.

In [5]:
r = rest_request_ambassador("example","seldon",API_AMBASSADOR)
print(r.text)

{
  "meta": {
    "puid": "n3r9g0frjb7l98k21oamov0il",
    "tags": {
    },
    "routing": {
    },
    "requestPath": {
      "classifier": "seldonio/mock_classifier:1.0"
    },
    "metrics": []
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.07517603988385778]
    }
  }
}


Check a REST request with the required header gets routed to the new model.

In [6]:
r = rest_request_ambassador("example","seldon",API_AMBASSADOR,headers={"location":"london"})
print(r.text)

{
  "meta": {
    "puid": "j7kuirsnel00kbi8qrhbpq2eb5",
    "tags": {
    },
    "routing": {
    },
    "requestPath": {
      "classifier": "seldonio/mock_classifier_rest:1.1"
    },
    "metrics": []
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.08067696978751894]
    }
  }
}


Now do the same checks with gRPC

In [3]:
grpc_request_ambassador("example","seldon",API_AMBASSADOR)

meta {
  puid: "7c6bl2f103bepqhfmc0kjp6h24"
  requestPath {
    key: "classifier"
    value: "seldonio/mock_classifier:1.0"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.07810444687073952
  }
}

In [4]:
grpc_request_ambassador("example","seldon",API_AMBASSADOR,headers={"location":"london"})

meta {
  puid: "vrgqcf91gaupgkrubnb1a7laff"
  requestPath {
    key: "classifier"
    value: "seldonio/mock_classifier_rest:1.1"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.09321255458350249
  }
}